In [ ]:
import tensorflow as tf
import numpy as np 
from PIL import Image

model = tf.saved_model.load('/kaggle/input/resnet50-retinal-disease-opacity-detection/ResNet50')
classes = [ "0" ,  "1" , ]

In [ ]:
import pandas as pd
train=pd.read_csv('/kaggle/input/retinal-disease-opacity-detection/train.csv')
train.head()

In [ ]:
train['filename']='/kaggle/input/retinal-disease-opacity-detection/train/'+train['opacity'].astype('str')+'/'+train['filename']
train.head()

In [ ]:
result=[]
for i in train.filename:
    img = Image.open(i).convert('RGB')
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)
    inp_numpy = np.array(img)[None]
    inp = tf.constant(inp_numpy, dtype='float32')
    class_scores = model(inp)[0].numpy()
    result.append(classes[class_scores.argmax()])
result[:5]

In [ ]:
train=train.assign(prediction=result)
train['prediction']=train['prediction'].astype('int')
train.head()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(train['opacity'],train['prediction']))

In [ ]:
test=pd.read_csv('/kaggle/input/retinal-disease-opacity-detection/test.csv')
test.head()

In [ ]:
test['filename']='/kaggle/input/retinal-disease-opacity-detection/test/'+test['filename']
test.head()

In [ ]:
result=[]
for i in test.filename:
    img = Image.open(i).convert('RGB')
    img = img.resize((300, 300 * img.size[1] // img.size[0]), Image.ANTIALIAS)
    inp_numpy = np.array(img)[None]
    inp = tf.constant(inp_numpy, dtype='float32')
    class_scores = model(inp)[0].numpy()
    result.append(classes[class_scores.argmax()])
result[:5]

In [ ]:
test=test.assign(opacity=result)
test['opacity']=test['opacity'].astype('int')
test.head()

In [ ]:
test.to_csv('./test.csv',index=False)